# Generage df_purchases_value

This notebook is the equivalent to the `generage_df_purchases_value.py` script that should be in the same folder. It parses and processes new rows in `df_purchases` to generate all the features and add a value to each purchase. It is kept here just for testing purposes.

Check also `generate_df_purchases_value_old.ipynb`, the original notebook where the first exploration and parsing of the data was performed, including much more detailed explanations for each field.

In [1]:
import pandas as pd
import numpy as np
import time # For timing executions
import pytz # For localization
import json # Reading / Saving dicts to json

In [2]:
df_purchases = pd.read_pickle('./lootbox_addiction/processed_dataframes/df_purchases.pkl')
df_purchases_value = pd.read_pickle('./lootbox_addiction/processed_dataframes/df_purchases_value.pkl')
df_src = pd.read_pickle('./lootbox_addiction/lootbox_db_scraping/df_pickles/df_src.pkl')
df_out = pd.read_pickle('./lootbox_addiction/lootbox_db_scraping/df_pickles/df_out.pkl')

In [3]:
display(df_purchases.tail(5))
display(df_purchases.info())
display(df_purchases_value.tail(5))
display(df_purchases_value.info())

,datetime,timestamp,user,src,out,time
1627658,2023-02-17 15:32:55,1676647975,S9***-CHVJ,左轮武器箱,沙漠之鹰 | 科林斯遗产,1676647975
1627659,2023-02-17 15:32:55,1676647975,SN***-FPTL,2022年里约热内卢锦标赛传奇组亲笔签名胶囊,印花 | Snappi | 2022年里约热内卢锦标赛,1676647975
1627660,2023-02-17 15:32:55,1676647975,AB***-NKVA,2022年里约热内卢锦标赛挑战组印花胶囊,印花 | Vitality | 2022年里约热内卢锦标赛,1676647975
1627661,2023-02-17 15:32:55,1676647975,SL***-86PQ,光环胶囊,印花 | 烂钱,1676647975
1627662,2023-02-17 15:32:55,1676647975,A3***-8XNE,2022年里约热内卢锦标赛传奇组印花胶囊,印花 | Team Spirit | 2022年里约热内卢锦标赛,1676647975


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1610463 entries, 0 to 1627662
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   datetime   1610463 non-null  object
 1   timestamp  1610463 non-null  int64 
 2   user       1610463 non-null  object
 3   src        1610463 non-null  object
 4   out        1610463 non-null  object
 5   time       1610463 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 86.0+ MB


None

,datetime_zh,timestamp,user,src,src_en,src_type,src_value,out,out_1_nopar,out_1_par,out_2_nopar,out_2_par_1,out_2_par_2,out_3,out_type,out_value
1596855,2023-02-17 11:17:28,1676603848,SS***-J2HQ,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | MOUZ | 2022年里约热内卢锦标赛,印花,,MOUZ,,,2022年里约热内卢锦标赛,Tournament Stickers,0.10
1596856,2023-02-17 11:17:28,1676603848,SF***-VXSA,2022年里约热内卢锦标赛传奇组印花胶囊,Rio 2022 Legends Sticker Capsule,Sticker Capsules,0.95,印花 | FaZe Clan | 2022年里约热内卢锦标赛,印花,,FaZe Clan,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
1596857,2023-02-17 11:17:29,1676603849,AV***-KUVJ,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | OG | 2022年里约热内卢锦标赛,印花,,OG,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
1596858,2023-02-17 11:17:29,1676603849,S5***-QLVL,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | Vitality | 2022年里约热内卢锦标赛,印花,,Vitality,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
1596859,2023-02-17 11:17:29,1676603849,SM***-LDZJ,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | OG（闪耀）| 2022年里约热内卢锦标赛,印花,,OG,闪耀,,2022年里约热内卢锦标赛,Tournament Stickers,0.03


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596860 entries, 0 to 1596859
Data columns (total 16 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   datetime_zh  1596860 non-null  datetime64[ns]
 1   timestamp    1596860 non-null  int64         
 2   user         1596860 non-null  object        
 3   src          1596860 non-null  object        
 4   src_en       1580576 non-null  object        
 5   src_type     1552423 non-null  object        
 6   src_value    1551412 non-null  float64       
 7   out          1596860 non-null  object        
 8   out_1_nopar  1596860 non-null  object        
 9   out_1_par    1596860 non-null  object        
 10  out_2_nopar  1596860 non-null  object        
 11  out_2_par_1  1596860 non-null  object        
 12  out_2_par_2  1596860 non-null  object        
 13  out_3        1596860 non-null  object        
 14  out_type     1596860 non-null  object        
 15  out_value    15

None

In [4]:
# Extract just the new rows in the df_purchases not yet present in df_purchases_value
def get_new_rows(df_purchases, df_purchases_value):
    merged_df = pd.merge(df_purchases, df_purchases_value, on=['timestamp', 'user'], how='inner')
    unique_df = df_purchases[~df_purchases[['timestamp', 'user']].apply(tuple, axis=1).isin(merged_df[['timestamp', 'user']].apply(tuple, axis=1))]
    return unique_df

In [5]:
# Process the new rows

def parse_new_values(new_rows):
    # Drop datetime and time columns, they are redundant
    print(f"Are the 'timestamp' and 'time' columns equal? {new_rows['time'].equals(new_rows['timestamp'])}")
    new_rows = new_rows.drop(columns=['datetime', 'time'])
    
    # Convert timestamp to datetimeUTC i datetime_zh
    new_rows['datetimeUTC'] = pd.to_datetime(new_rows['timestamp'], unit='s') # Will be in the UTC timezone by default
    new_rows['datetime_zh'] = new_rows['datetimeUTC'].dt.tz_localize("UTC").dt.tz_convert("Asia/Shanghai").dt.tz_localize(None)
    
    # Sort by purchase datetime
    new_rows.sort_values(by='timestamp', inplace=True)
    
    # Parse, 1st stage
    
    # Split the `out` column of the lootbox dataset into three different features
    new_rows[['out_1', 'out_2', 'out_3']] =  new_rows['out'].str.split("|", expand=True)
    
    # Let's separate whatever is within parenthesis as new features
    #the u prefix is to specify that the string is a Unicode string. e.g. 截短霰弹枪（纪念品） 
    new_rows['out_1_par'] = new_rows['out_1'].str.strip().str.extract(u'（(.*?)）')
    new_rows['out_2_par'] = new_rows['out_2'].str.strip().str.extract(r'（([^（）]*)）$')
    
    # Also, let's put the part with no parentheses in another column
    new_rows['out_1_nopar'] = new_rows['out_1'].str.replace(r'（[^（）]*）(?=[^（）]*$)', '', regex=True)
    new_rows['out_2_nopar'] = new_rows['out_2'].str.replace(r'（[^（）]*）(?=[^（）]*$)', '', regex=True)
    
    # out_2_par has commas, so the column can be splitted into more features
    new_rows['out_2_par'] = new_rows['out_2_par'].fillna('') # If all the rows contain out_2_par as Nan, it fails, so convert to empty string
    new_rows[['out_2_par_1', 'out_2_par_2']] =  new_rows['out_2_par'].str.split("，", expand=True)
    
    # Rearrange the features and convert the Nan to empty strings
    new_rows = new_rows[['datetime_zh', 'timestamp', 'user', 'src', 'out', 'out_1_nopar', 'out_1_par', 'out_2_nopar', 'out_2_par_1', 'out_2_par_2', 'out_3']]
    new_rows = new_rows.fillna('')
    
    # sort and reset index
    new_rows.sort_values(by='timestamp', inplace=True)
    new_rows.reset_index(drop=True, inplace=True)
    
    return new_rows

In [6]:
# Complements new rows with info from df_src
def get_src_info(new_rows, df_src):
    
    # Load dictionary for manually correcting some missing values
    with open('./lootbox_addiction/lootbox_db_scraping/lootboxes_zh_en.json', 'r', encoding='utf-8') as file:
        lootboxes_zh_en = json.load(file)
    # Generate dict, but opposite direction (English->Chinese)
    lootboxes_en_zh = {v: k for k, v in lootboxes_zh_en.items()}
    
    # Now map the values of df_src into the df_purchases (merge both dataframes)
    merged_df = new_rows.merge(df_src, left_on='src', right_on='lootbox_zh', how='left')
    new_rows["src_en"] = merged_df["lootbox_en"] # this is not being applied for some reason 
    new_rows['src_en'] = new_rows['src'].map(lootboxes_zh_en).fillna(new_rows['src_en']) # Manually fill missing values from the dict. is this still necessary?
    
    
    # Add the type of lootbox into df_purchases
    merged_df = new_rows.merge(df_src, left_on='src_en', right_on='lootbox_en', how='left')
    new_rows["src_type"] = merged_df["Type_en"]
    
    # Manually add some type of lootbox that wasn't done automatically
    new_rows["src_type"] = np.where((new_rows['src_en'].str.contains('Sticker Capsule')) & (new_rows['src_type'].isnull()), "Sticker Capsules", new_rows["src_type"]) # e.g. 10 Year Birthday Sticker Capsule
    new_rows["src_type"] = np.where((new_rows['src_en'].str.contains('Case')) & (new_rows['src_type'].isnull()), "Cases", new_rows["src_type"]) # e.g.: Falchion case
    new_rows["src_type"] = np.where((new_rows['src_en'].str.contains('Music Kit')) & (new_rows['src_type'].isnull()), "Music Kit Boxes", new_rows["src_type"]) # e.g.: Initiators Music Kit Box

    
    # Add the price to each lootbox (some of them manually)
    new_rows['src_value'] = merged_df["Value"]
    new_rows['src_value'] = np.where((new_rows['src_type'] == 'Music Kit Boxes') | (new_rows['src_type'] == 'Graffiti Capsules'), merged_df['Value'], np.nan)
    new_rows['src_value'] = np.where(((new_rows['src_type'] == 'Cases') & (new_rows['src_value'].isnull())), 2.5, new_rows['src_value']) # Cases
    new_rows['src_value'] = np.where(((new_rows['src_type'] == 'Souvenir Packages') & (new_rows['src_value'].isnull())), 3.0, new_rows['src_value']) # Souvenir packages
    new_rows['src_value'] = np.where(((new_rows['src_type'] == 'Patch Packs') & (new_rows['src_value'].isnull())), 2.0, new_rows['src_value']) # Patch packs
    new_rows['src_value'] = np.where(((new_rows['src_type'] == 'Pins Capsules') & (new_rows['src_value'].isnull())), 9.49, new_rows['src_value']) # Pin capsules
    new_rows['src_value'] = np.where(((new_rows['src_type'] == 'Sticker Capsules') & (new_rows['src_value'].isnull())), 0.95, new_rows['src_value']) # Sticker capsules

    # Convert values like $ 1.21 to 1.21 (float) and convert whole column to float
    new_rows['src_value'] = new_rows['src_value'].apply(lambda x: x.strip('$') if isinstance(x, str) and "$" in x else x)
    new_rows['src_value']= pd.to_numeric(new_rows['src_value'])
    
    # Rearrange column in df_purchases so src_en goes after src
    new_rows = new_rows[['datetime_zh', 'timestamp', 'user', 'src', 'src_en', 'src_type', 'src_value', 'out', 'out_1_nopar',
       'out_1_par', 'out_2_nopar', 'out_2_par_1', 'out_2_par_2', 'out_3']]
    
    # remove temporary variables
    del merged_df
    
    return new_rows

In [7]:
# This functions returns the type and value of `out` for the df_purchases, by checking the df_out dataframe
# (this function should be rewriten so it can use column names instead of indices)
def get_value(purchase, verbose=False):
    outcategory = ''
    stripped = [s.strip() for s in purchase[8:]]
    purchase = purchase[:8] + stripped
    if verbose: print("Purchase: ", purchase)

    # Some specific cases to deal with manually
    if purchase[8] == 'CZ75': purchase[8] = 'CZ75 自动手枪' # The pistol CZ75 appears as CZ75 自动手枪 in df_out. It's an exception that needs to be corrected manually.
    if purchase[8] == 'M4A1 消音型': purchase[8] = 'M4A1 消音版' # The weapon M4A1 消音型 appears as M4A1 消音版 in df_out.

    if purchase[8] == '印花': # If it's a sticker
        # It still fails if a sticker, patch and graffiti have the same name (finds more than 1 result, like item 580721), we have to control those cases
        if verbose: print("It's a sticker")
        outcategory = "Regular Stickers"
        if verbose: print(f"Name of sticker {purchase[10]}")
        if verbose: print(f"Grade of the sticker: {purchase[11]}")
        #if purchase[10] == "冠军": # Champion
        #    print("Champion!")
        if purchase[13] != "": # If there's something in out3, it's a tournament sticker
            outcategory = "Tournament Sticker"
            if verbose: print("It's a tournament sticker")
            if verbose: print(f"It belongs to the tournament {purchase[13]}")
            df_query = df_out.query("Weapon_zh == @purchase[10] & Skin_Name_zh == @purchase[13]")
        else:
            if verbose: print("It's a non-tournament sticker")
            df_query = df_out.query("Type_zh == '普通贴纸' & Weapon_zh == @purchase[10] & Skin_Name_zh == @purchase[13]") # to separate them from graffiti and patches

        value =  df_query['Value']
        if verbose: display(df_query)
    

    elif purchase[8] == '音乐盒':
        if verbose: print("It's a music kit")
        df_query = df_out.query("Type_en == 'Music Kits' & Weapon_zh == @purchase[8]")
        outcategory = "Music Kits"
        value =  df_query['Value']

    elif '★' in purchase[9]:
        if verbose: print("Item with a star! ★")
        searchitem = purchase[8]+'（'+purchase[9]+'）'
        if verbose: print(searchitem)
        df_query = df_out.query("Weapon_zh == @searchitem & Skin_Name_zh == @purchase[10]")
        value =  df_query['Value']
        #display(df_query)

    else:
        if verbose: print("It's likely a weapon skin") # Still fails for music boxes
        #df_weaponsearch = df_out[df_out['Weapon_zh'] == purchase[6]]
        
        df_query = df_out.query("Skin_Name_zh == @purchase[10] & Weapon_zh == @purchase[8] ") # Control if 0 cases are returned, like item with timestamp 1673442333
        outcategory = "Unknown Weapon skin"
        if purchase[9] == '纪念品':
            if verbose: print("It's a Souvenir weapon.")
            value =  df_query['Value_Souvenir']
        elif purchase[9] == 'StatTrak™':
            if verbose: print("It's a StatTrak weapon.")
            value =  df_query['Value_Stattrak']
        elif purchase[9] == '':
            if verbose: print("The weapon has the grade Normal.")
            value =  df_query['Value']

        else:
            if verbose: print("What is this?") # if anything else fails
            return 'not found', np.nan
        if verbose: display(df_query)

    # Parse value (will be in '$ 34 - $ 56' format)
    if verbose: print("value: ", value)
    
    if len(value.index) == 0: # If no results were found, set the value as np.nan
        if verbose: print("No results found")
        if outcategory == "":
            return 'unknown', np.nan 
        else:
            return outcategory, np.nan # sometimes we know the type, even if it was not found
    elif len(value.index) > 1: 
        value = value.head(1) # If more than 1 value is returned, keep the first one. Some rows are repeated in df_out, it's fine
        
    value = value.item()

    if '-' in value: value = value.split(' - ')[0] # If it's a range, get the first value (the lowest)
    if verbose: print("value: ", value)
    value = value.replace('$', '').strip() # remove the $ sign
    if verbose: print("value without $: ", value)
    return df_query.iloc[0]['Type_en'], value # Returns a 2-element tuple with the type of out and its value

In [8]:
def get_out_info(new_rows):
    

    from tqdm import tqdm # to show a progress bar. Might make it a bit slower.
    tqdm.pandas()
    
    #df_purchases_value = df_purchases.iloc[-1000:]
    df_purchases_value = new_rows.copy()
    df_purchases_value[['out_type', 'out_value']] = df_purchases_value.progress_apply(lambda row: pd.Series(get_value(list(row), verbose=False)), axis=1)
    #df_purchases_value[['out_type', 'out_value']] = df_purchases_value.apply(lambda row: pd.Series(get_value(list(row), verbose=False)), axis=1)
    
    return df_purchases_value

In [9]:
# Gets new rows as df_purchases_new
st = time.time()
df_purchases_new = get_new_rows(df_purchases, df_purchases_value)
print(df_purchases_new.shape)
et = time.time()
print('Execution time:', et - st, 'seconds')

(13600, 6)
Execution time: 19.145190477371216 seconds


In [11]:
# Parses new columns in the new rows
st = time.time()
df_purchases_new = parse_new_values(df_purchases_new)
display(df_purchases_new.head(5))
et = time.time()
print('Execution time:', et - st, 'seconds')

Are the 'timestamp' and 'time' columns equal? True


,datetime_zh,timestamp,user,src,out,out_1_nopar,out_1_par,out_2_nopar,out_2_par_1,out_2_par_2,out_3
0,2023-02-17 11:26:37,1676604397,A5***-VDUJ,CS:GO布章包,布章 | 血猎,布章,,血猎,,,
1,2023-02-17 11:26:38,1676604398,SG***-E9VL,梦魇武器箱,P2000 | 升天,P2000,,升天,,,
2,2023-02-17 11:26:39,1676604399,S5***-7WNE,2022年里约热内卢锦标赛炙热沙城 II 纪念包,P90（纪念品） | 沙漠 DDPAT,P90,纪念品,沙漠 DDPAT,,,
3,2023-02-17 11:26:40,1676604400,S3***-JDQE,2022年里约热内卢锦标赛挑战组亲笔签名胶囊,印花 | max | 2022年里约热内卢锦标赛,印花,,max,,,2022年里约热内卢锦标赛
4,2023-02-17 11:26:40,1676604400,SQ***-LJXL,反冲武器箱,MAC-10 | 萌猴迷彩,MAC-10,,萌猴迷彩,,,


Execution time: 0.16886544227600098 seconds


In [12]:
# Fills new rows with df_src data (src_en, src_type and src_value)
st = time.time()
df_purchases_new = get_src_info(df_purchases_new, df_src)
display(df_purchases_new.sample(2))
et = time.time()
print('Execution time:', et - st, 'seconds')

,datetime_zh,timestamp,user,src,src_en,src_type,src_value,out,out_1_nopar,out_1_par,out_2_nopar,out_2_par_1,out_2_par_2,out_3
4347,2023-02-17 15:18:49,1676618329,AJ***-AWVJ,蛇噬武器箱,Snakebite Case,Cases,2.50,CZ75 | 短趾雕,CZ75,,短趾雕,,,
140,2023-02-17 11:27:14,1676604434,SY***-TCWC,2022年里约热内卢锦标赛竞争组印花胶囊,Rio 2022 Contenders Sticker Capsule,Sticker Capsules,0.95,印花 | 00 Nation | 2022年里约热内卢锦标赛,印花,,00 Nation,,,2022年里约热内卢锦标赛


Execution time: 0.07808732986450195 seconds


In [13]:
# Test the function with a sample in of df_purchaes
get_value(['2023-02-04 08:07:33', 1675469253, 'AU***-T3DG', '梦魇武器箱', 'Dreams & Nightmares Case', 'Cases', 2.5, 'P2000 | 升天', 'P2000 ', '', ' 升天', '', '', ''])
print(list(pd.Series(df_purchases_new.iloc[0])))
get_value(list(pd.Series(df_purchases_new.iloc[0])), verbose=True)

[Timestamp('2023-02-17 11:26:37'), 1676604397, 'A5***-VDUJ', 'CS:GO布章包', 'CS:GO Patch Pack', 'Patch Packs', 2.0, '布章 | 血猎', '布章 ', '', ' 血猎', '', '', '']
Purchase:  [Timestamp('2023-02-17 11:26:37'), 1676604397, 'A5***-VDUJ', 'CS:GO布章包', 'CS:GO Patch Pack', 'Patch Packs', 2.0, '布章 | 血猎', '布章', '', '血猎', '', '', '']
It's likely a weapon skin
The weapon has the grade Normal.


,index,Type_en,Type_zh,Weapon_en,Weapon_zh,Image,Link,Skin_Name,Skin_Name_zh,Grade,Rarity,Value,Value_Stattrak,Value_Souvenir,Skin_Link,Found_in,Found_in_zh,Found_in_Link,Found_in_Link_zh,Link_zh


value:  Series([], Name: Value, dtype: object)
No results found


('Unknown Weapon skin', nan)

In [14]:
# Fills new rows with df_out data 
print(df_purchases_new.info())
st = time.time()
df_purchases_new = get_out_info(df_purchases_new)
et = time.time()
display(df_purchases_new.sample(3))
print('Execution time:', et - st, 'seconds')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13600 entries, 0 to 13599
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime_zh  13600 non-null  datetime64[ns]
 1   timestamp    13600 non-null  int64         
 2   user         13600 non-null  object        
 3   src          13600 non-null  object        
 4   src_en       11707 non-null  object        
 5   src_type     11437 non-null  object        
 6   src_value    11416 non-null  float64       
 7   out          13600 non-null  object        
 8   out_1_nopar  13600 non-null  object        
 9   out_1_par    13600 non-null  object        
 10  out_2_nopar  13600 non-null  object        
 11  out_2_par_1  13600 non-null  object        
 12  out_2_par_2  13600 non-null  object        
 13  out_3        13600 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(11)
memory usage: 1.5+ MB
None


100%|████████████████████████████████████████████████████████████████████████████████| 13600/13600 [06:27<00:00, 35.13it/s]


,datetime_zh,timestamp,user,src,src_en,src_type,src_value,out,out_1_nopar,out_1_par,out_2_nopar,out_2_par_1,out_2_par_2,out_3,out_type,out_value
10733,2023-02-17 21:04:24,1676639064,AB***-Z7TN,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | Cloud9 | 2022年里约热内卢锦标赛,印花,,Cloud9,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
1843,2023-02-17 13:01:54,1676610114,AJ***-URVC,2022年里约热内卢锦标赛传奇组亲笔签名胶囊,Rio 2022 Legends Autograph Capsule,Sticker Capsules,0.95,印花 | YEKINDAR | 2022年里约热内卢锦标赛,印花,,YEKINDAR,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
5072,2023-02-17 15:59:10,1676620750,S9***-ACTA,反冲武器箱,Recoil Case,Cases,2.50,法玛斯 | 喵喵36,法玛斯,,喵喵36,,,,Rifles,0.08


Execution time: 387.1849527359009 seconds


In [24]:
display(df_purchases_new.info())
display(df_purchases_value.info())
print("Valid new data %", df_purchases_new['out_value'].count()/df_purchases_new['out'].count()*100)
print("Valid values old data %", df_purchases_value['out_value'].count()/df_purchases_value['out'].count()*100)

# (df_purchases_value already includes df_purchases_new... i accidentally merged them)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13600 entries, 0 to 13599
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime_zh  13600 non-null  datetime64[ns]
 1   timestamp    13600 non-null  int64         
 2   user         13600 non-null  object        
 3   src          13600 non-null  object        
 4   src_en       11707 non-null  object        
 5   src_type     11437 non-null  object        
 6   src_value    11416 non-null  float64       
 7   out          13600 non-null  object        
 8   out_1_nopar  13600 non-null  object        
 9   out_1_par    13600 non-null  object        
 10  out_2_nopar  13600 non-null  object        
 11  out_2_par_1  13600 non-null  object        
 12  out_2_par_2  13600 non-null  object        
 13  out_3        13600 non-null  object        
 14  out_type     13600 non-null  object        
 15  out_value    11563 non-null  object        
dtypes: d

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596860 entries, 0 to 1596859
Data columns (total 16 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   datetime_zh  1596860 non-null  datetime64[ns]
 1   timestamp    1596860 non-null  int64         
 2   user         1596860 non-null  object        
 3   src          1596860 non-null  object        
 4   src_en       1580576 non-null  object        
 5   src_type     1552423 non-null  object        
 6   src_value    1551412 non-null  float64       
 7   out          1596860 non-null  object        
 8   out_1_nopar  1596860 non-null  object        
 9   out_1_par    1596860 non-null  object        
 10  out_2_nopar  1596860 non-null  object        
 11  out_2_par_1  1596860 non-null  object        
 12  out_2_par_2  1596860 non-null  object        
 13  out_3        1596860 non-null  object        
 14  out_type     1596860 non-null  object        
 15  out_value    15

None

Valid new data % 85.0220588235294
Valid values old data % 97.81690317247599


In [25]:
# Only 309763 purchases have value, out of 349318 (88.5%, should be higher), in the prevoius data we reached 95.9%


#display(df_purchases_new[missingvaluesmask].sample(10))

# Most commonly missed out values in new rows
missingvaluesmask = df_purchases_new['out_value'].isnull()
display(df_purchases_new[missingvaluesmask][['datetime_zh', 'out']].groupby('out').count().sort_values(by='datetime_zh', ascending=False).head(10))

# Most commonly missed out values in new rows
missingvaluesmask = df_purchases_value['out_value'].isnull()
display(df_purchases_value[missingvaluesmask][['datetime_zh', 'out']].groupby('out').count().sort_values(by='datetime_zh', ascending=False).head(10))

,datetime_zh
out,
印花 | 蛰刺如蝶,90
印花 | 龟派鸡功,88
印花 | 神之投手,87
印花 | 6,85
印花 | 吊打,83
印花 | 信手拈刀,82
印花 | 小T快润,79
印花 | 疯狂星期一,79
印花 | 不是机器人,75


,datetime_zh
out,
双持贝瑞塔 | 通风机,2053
格洛克 18 型（StatTrak™） | 一目了然,754
音乐盒 | Juelz - 神枪手,745
印花 | sk0R | 2022年里约热内卢锦标赛,735
音乐盒 | Sullivan King - 困兽,733
音乐盒 | Knock2 - 冲击星*,727
音乐盒 | Humanity's Last Breath - 虚空,697
音乐盒 | Meechy Darko - 哥特浮华,696
音乐盒 | 3kliksphilip - 追溯起源,690


In [26]:
# Manually insert prices to 'out_value' based on 'out' from dictionary
with open('./lootbox_addiction/lootbox_db_scraping/manual_prices.json', 'r', encoding='utf-8') as file:
        manual_prices = json.load(file)
        
display(manual_prices)

df_purchases_new['out_value'] = df_purchases_new['out'].map(manual_prices).fillna(df_purchases_new['out_value'])
df_purchases_value['out_value'] = df_purchases_value['out'].map(manual_prices).fillna(df_purchases_value['out_value'])

{'印花 | Grayhound Gaming | 2022年里约热内卢锦标赛': 0.03,
 '印花 | GamerLegion | 2022年里约热内卢锦标赛': 0.03,
 '印花 | 00 Nation | 2022年里约热内卢锦标赛': 0.32,
 '格洛克 18 型 | 一目了然': 0.16,
 '印花 | 00 Nation（闪耀）| 2022年里约热内卢锦标赛': 1.76,
 '印花 | GamerLegion（闪耀）| 2022年里约热内卢锦标赛': 0.08,
 '印花 | Grayhound Gaming（闪耀）| 2022年里约热内卢锦标赛': 0.05,
 '加利尔 AR | 战吼斑纹': 0.29,
 '印花 | 360 No Scope': 0.08,
 '印花 | Zap Cat': 0.13,
 '印花 | Fireball': 0.1,
 '印花 | Explosive Strength': 0.11,
 "印花 | Knife's Edge": 0.08,
 '印花 | Not A Bot': 0.3,
 '印花 | Batter Up': 0.06,
 '印花 | Aim And Fire': 0.06,
 '印花 | Sting Like A Butterly': 0.07,
 '印花 | Well Played': 0.05,
 '印花 | Run T, Run': 0.17,
 '印花 | Spectators': 0.11,
 'SCAR-20 | Fragments': 0.61,
 'Tec-9 | Rebel': 0.53,
 'SG 553 | Cyberforce': 0.61,
 'P250 | Re.built': 0.58,
 'MAG-7 | Insomnia': 0.59,
 'MP9 | Featherweight': 0.55,
 'MP5-SD | Liquidation': 0.66}

In [27]:
df_purchases_new[df_purchases_new['out']== "印花 | GamerLegion | 2022年里约热内卢锦标赛"].head(3)

,datetime_zh,timestamp,user,src,src_en,src_type,src_value,out,out_1_nopar,out_1_par,out_2_nopar,out_2_par_1,out_2_par_2,out_3,out_type,out_value
88,2023-02-17 11:26:59,1676604419,S5***-NEVL,2022年里约热内卢锦标赛竞争组印花胶囊,Rio 2022 Contenders Sticker Capsule,Sticker Capsules,0.95,印花 | GamerLegion | 2022年里约热内卢锦标赛,印花,,GamerLegion,,,2022年里约热内卢锦标赛,Tournament Sticker,0.03
126,2023-02-17 11:27:08,1676604428,A6***-56UA,2022年里约热内卢锦标赛竞争组印花胶囊,Rio 2022 Contenders Sticker Capsule,Sticker Capsules,0.95,印花 | GamerLegion | 2022年里约热内卢锦标赛,印花,,GamerLegion,,,2022年里约热内卢锦标赛,Tournament Sticker,0.03
498,2023-02-17 11:50:26,1676605826,A2***-KAWG,2022年里约热内卢锦标赛竞争组印花胶囊,Rio 2022 Contenders Sticker Capsule,Sticker Capsules,0.95,印花 | GamerLegion | 2022年里约热内卢锦标赛,印花,,GamerLegion,,,2022年里约热内卢锦标赛,Tournament Sticker,0.03


In [461]:
# Finally, concat processed new rows to df_purchases_value
df_purchases_value = pd.concat([df_purchases_value, df_purchases_new])

display(df_purchases_value.info())

# Sort i Reset index ??
df_purchases_value.sort_values(by='timestamp', inplace=True)
df_purchases_value.reset_index(drop=True, inplace=True)

# Check for duplicate
df_purchases_value = df_purchases_value.drop_duplicates(
                          subset = ['timestamp', 'user'],
                          keep = 'last').reset_index(drop = True)

# Save to pickle, overwritting the last file
df_purchases_value.to_pickle('./lootbox_addiction/processed_dataframes/df_purchases_value.pkl')

,datetime_zh,timestamp,user,src,src_en,src_type,src_value,out,out_1_nopar,out_1_par,out_2_nopar,out_2_par_1,out_2_par_2,out_3,out_type,out_value
349313,2023-02-17 11:17:28,1676603848,AC***-R6YG,2022年里约热内卢锦标赛冠军亲笔签名胶囊,Rio 2022 Champions Autograph Capsule,Sticker Capsules,0.95,印花 | fame（冠军）| 2022年里约热内卢锦标赛,印花,,fame,冠军,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
349314,2023-02-17 11:17:28,1676603848,SF***-VXSA,2022年里约热内卢锦标赛传奇组印花胶囊,Rio 2022 Legends Sticker Capsule,Sticker Capsules,0.95,印花 | FaZe Clan | 2022年里约热内卢锦标赛,印花,,FaZe Clan,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
349315,2023-02-17 11:17:29,1676603849,S5***-QLVL,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | Vitality | 2022年里约热内卢锦标赛,印花,,Vitality,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
349316,2023-02-17 11:17:29,1676603849,AV***-KUVJ,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | OG | 2022年里约热内卢锦标赛,印花,,OG,,,2022年里约热内卢锦标赛,Tournament Stickers,0.03
349317,2023-02-17 11:17:29,1676603849,SM***-LDZJ,2022年里约热内卢锦标赛挑战组印花胶囊,Rio 2022 Challengers Sticker Capsule,Sticker Capsules,0.95,印花 | OG（闪耀）| 2022年里约热内卢锦标赛,印花,,OG,闪耀,,2022年里约热内卢锦标赛,Tournament Stickers,0.03


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1596863 entries, 0 to 349317
Data columns (total 16 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   datetime_zh  1596863 non-null  datetime64[ns]
 1   timestamp    1596863 non-null  int64         
 2   user         1596863 non-null  object        
 3   src          1596863 non-null  object        
 4   src_en       1580579 non-null  object        
 5   src_type     1552426 non-null  object        
 6   src_value    1551415 non-null  float64       
 7   out          1596863 non-null  object        
 8   out_1_nopar  1596863 non-null  object        
 9   out_1_par    1596863 non-null  object        
 10  out_2_nopar  1596863 non-null  object        
 11  out_2_par_1  1596863 non-null  object        
 12  out_2_par_2  1596863 non-null  object        
 13  out_3        1596863 non-null  object        
 14  out_type     1596863 non-null  object        
 15  out_value    156

None